<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/code/RBD_Intersectionality_IAT_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [128]:
import pandas as pd #for dealing with csv import
import os # for joining paths and filenames sensibly
import numpy as np #for the population std
import glob # for finding csv data files
import platform # paths use different dividers on linux vs windows, so we need to test for this
import string

## Define Utility functions

In [129]:
#Define Utility functions
def adjustedmean(RTs,corrs,penalty):
    n=len(corrs) #trials
    n_errors=int(n-sum(corrs)) #errors
    print("Number of correctness: ", int(sum(corrs)))
    print("Number of errors: ", n_errors)
    cor_RTs=np.array(corrs)*RTs #sum of correct RTs
    cor_mean=sum(cor_RTs)/sum(corrs)
    
    #mean with errors replaced with penalty value
    return cor_mean+(n_errors*penalty)/n



def exclude_slows(RTs,corrs,slowRT_limit):
    new_rt=[] #holding variables
    new_cr=[]
    for i in range(len(RTs)): #iterate over every item
        if RTs[i] < slowRT_limit: #if it isn't too fast, include RT and corr values
            new_rt.append(RTs[i])
            new_cr.append(corrs[i])
    
    return (new_rt, new_cr)

## Load data

In [130]:
df_IAT_BM = pd.read_csv ('https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/data/IAT_BM.csv', header=0)
df_IAT_WM = pd.read_csv ('https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/data/IAT_WM.csv', header=0)

##Specify thresholds

In [131]:
penalty=0.600 #penalty - in seconds - for incorrect responses
slowRT_limit=1200 #threshold at which slow RTs are discarded
fastRT_limit=300 #threshold which defines responses which are "too fast"
fast_prop_limit=0.1 # threshold proportion of "too fast" responses which defines exclusion of ppt


## Data extraction

In [132]:
df_IAT_BM.rename(columns={"Spreadsheet: metadata": "Congruence", "allocator-k3xu": "Group Type"}, inplace = True)
df_IAT_BM_2 = df_IAT_BM.drop(["Spreadsheet: ImageLeft", "Spreadsheet: ImageRight"], axis = 1)
df_IAT_WM.rename(columns={"Spreadsheet: metadata": "Congruence", "allocator-k3xu": "Group Type"}, inplace = True)
df_IAT_WM_2 = df_IAT_WM.drop(["Spreadsheet: ImageLeft", "Spreadsheet: ImageRight"], axis = 1)
# df_IAT_BM_2
# df_IAT_WM_2

df_IAT_BM_STR = df_IAT_BM_2[((df_IAT_BM_2["Spreadsheet: TextLeft"] == "Strong") | (df_IAT_BM_2["Spreadsheet: TextRight"] == "Strong")) & (~df_IAT_BM_2["Congruence"].str.startswith("practice"))]
# print(df_IAT_BM_STR.shape)
df_IAT_BM_INT = df_IAT_BM_2[((df_IAT_BM_2["Spreadsheet: TextLeft"] == "Intelligence") | (df_IAT_BM_2["Spreadsheet: TextRight"] == "Intelligence")) & (~df_IAT_BM_2["Congruence"].str.startswith("practice"))]
# print(df_IAT_BM_INT.shape)
df_IAT_WM_STR = df_IAT_WM_2[((df_IAT_WM_2["Spreadsheet: TextLeft"] == "Strong") | (df_IAT_WM_2["Spreadsheet: TextRight"] == "Strong")) & (~df_IAT_WM_2["Congruence"].str.startswith("practice"))]
# print(df_IAT_WM_STR.shape)
df_IAT_WM_INT = df_IAT_WM_2[((df_IAT_BM_2["Spreadsheet: TextLeft"] == "Intelligence") | (df_IAT_WM_2["Spreadsheet: TextRight"] == "Intelligence")) & (~df_IAT_WM_2["Congruence"].str.startswith("practice"))]
# print(df_IAT_WM_INT.shape)
congr_corr_BM_STR = df_IAT_BM_STR[df_IAT_BM_STR["Congruence"] == "congruent"]["Correct"].dropna().tolist()
congr_rts_BM_STR = df_IAT_BM_STR[df_IAT_BM_STR["Congruence"] == "congruent"]["Absolute Reaction Time"].dropna().tolist()
incon_corr_BM_STR = df_IAT_BM_STR[df_IAT_BM_STR["Congruence"] == "incongruent"]["Correct"].dropna().tolist()
incon_rts_BM_STR = df_IAT_BM_STR[df_IAT_BM_STR["Congruence"] == "incongruent"]["Absolute Reaction Time"].dropna().tolist()
# check correctness list has the same length as the rt list 
print(len(congr_corr_BM_STR))
print(len(congr_rts_BM_STR))



2772
2772


## Analysis and calculation

In [133]:
# -----------ANALYSE / CALCULATE
        
#1 discard subject if too many fast responses
if sum(np.array(congr_rts_BM_STR + incon_rts_BM_STR)<fastRT_limit)>len(congr_rts_BM_STR + incon_rts_BM_STR)*fast_prop_limit:
    print ("excluding subject for BM STR because too many fast responses")
else:
    #2 Eliminate scores over 10,000 ms
            
    congr_rts,congr_corr=exclude_slows(congr_rts_BM_STR,congr_corr_BM_STR,slowRT_limit)              
    incon_rts,incon_corr=exclude_slows(incon_rts_BM_STR,incon_corr_BM_STR,slowRT_limit)
    
    #3 Calculate pooled std
    #pooled_std=pooled.std(0) #n-1 std sample std
    #(Use N not N-1 because this is the whole sample). 
    #numpy.std is population std
    pooled=congr_rts + incon_rts #all RTs from both blocks, correct and incorrect
    pooled_std=np.std(pooled)
    
    #4 Calculated adjusted means, including the penalty
    congr_adjmean=adjustedmean(congr_rts,congr_corr,penalty)
    incon_adjmean=adjustedmean(incon_rts,incon_corr,penalty)
    
    #5 Calculate the IAT, so that pro-stereotype RTs are a -ve score
    IAT=(congr_adjmean-incon_adjmean)/pooled_std
    
    simpleIAT=sum(congr_rts)/len(congr_rts)-sum(incon_rts)/len(incon_rts)        
    
    print("IAT for BM_STR" + " is : {:+.3f}".format(IAT))
    print("Mean difference (uncorrected) is {:+.3f}".format(simpleIAT)+" seconds")

Number of correctness:  2189
Number of errors:  234
Number of correctness:  2275
Number of errors:  179
IAT for BM_STR is : +0.119
Mean difference (uncorrected) is +21.111 seconds


## Deprecated functions and declarations

In [134]:
# #extract the stuff we're interested in (n.b i am indexing using the column names defined in the csv)
#     #dropna() drops nans
#     #tolist() converts from series to list
#     corrs=df['key_resp_9.corr'].dropna().tolist()
#     rts=df['key_resp_9.rt'].dropna().tolist()
#     block_length=int(len(corrs)/2)
#     #find order 
#     order=df['order'].tolist()[0]
#     #1 congr then incong
#     #2 incongr then congr    
#     if order==1:
#         congr_corr=corrs[0:block_length]
#         congr_rts=rts[0:block_length]
#         incon_corr=corrs[block_length:]
#         incon_rts=rts[block_length:]
#     else:
#         congr_corr=corrs[block_length:]
#         congr_rts=rts[block_length:]
#         incon_corr=corrs[0:block_length]
#         incon_rts=rts[0:block_length]
#     #1 discard subject if too many fast responses
#     if sum(np.array(congr_rts + incon_rts)<fastRT_limit)>len(congr_rts + incon_rts)*fast_prop_limit:
#         print "excluding subject for " + os.path.basename(filename) + " because too many fast responses"
#     else:
#         #2 Eliminate scores over 10,000 ms
             
#         congr_rts,congr_corr=exclude_slows(congr_rts,congr_corr,slowRT_limit)              
#         incon_rts,incon_corr=exclude_slows(incon_rts,incon_corr,slowRT_limit)
        
#         #3 Calculate pooled std
#         #pooled_std=pooled.std(0) #n-1 std sample std
#         #(Use N not N-1 because this is the whole sample). 
#         #numpy.std is population std
#         pooled=congr_rts + incon_rts #all RTs from both blocks, correct and incorrect
#         pooled_std=np.std(pooled)
        
#         #4 Calculated adjusted means, including the penalty
#         congr_adjmean=adjustedmean(congr_rts,congr_corr,penalty)
#         incon_adjmean=adjustedmean(incon_rts,incon_corr,penalty)
        
#         #5 Calculate the IAT, so that pro-stereotype RTs are a -ve score
#         IAT=(congr_adjmean-incon_adjmean)/pooled_std
        
#         simpleIAT=mean(congr_rts)-mean(incon_rts)        
        
#         print "IAT for " + os.path.basename(filename) + " is : {:+.3f}".format(IAT)
#         print "Mean difference (uncorrected) is {:+.3f}".format(simpleIAT)+" seconds"
# df_IAT_BM_INT_inin = df_IAT_BM_INT[df_IAT_BM_INT["Group Type"] == "Condition_InIn"]
# df_IAT_BM_INT_inoutGender = df_IAT_BM_INT[df_IAT_BM_INT["Group Type"] == "Condition_InOut-gender"]
# df_IAT_BM_INT_inoutRace = df_IAT_BM_INT[df_IAT_BM_INT["Group Type"] == "Condition_InOut-race"]
# df_IAT_BM_INT_inoutBoth = df_IAT_BM_INT[df_IAT_BM_INT["Group Type"] == "Condition_InOut-both"]
# df_IAT_WM_INT_inin = df_IAT_WM_INT[df_IAT_WM_INT["Group Type"] == "Condition_InIn"]
# df_IAT_WM_INT_inoutGender = df_IAT_WM_INT[df_IAT_WM_INT["Group Type"] == "Condition_InOut-gender"]
# df_IAT_WM_INT_inoutRace = df_IAT_WM_INT[df_IAT_WM_INT["Group Type"] == "Condition_InOut-race"]
# df_IAT_WM_INT_inoutBoth = df_IAT_WM_INT[df_IAT_WM_INT["Group Type"] == "Condition_InOut-both"]

## Reference: 
IAT data analysis script adapted from https://github.com/tomstafford/IAT

Calculate IAT score from data generated by Robin's PsychoPy script [TQS Feb 2014]

*Greenwald, A. G., Nosek, B. A., & Banaji, M. R. (2003). Understanding and using the implicit association test: I. An improved scoring algorithm. Journal of personality and social psychology, 85(2), 1972-216.*